<h2 style="color: white;
    font-size: 32px; 
    text-align: center; 
    background: #00cccc; 
    padding: 30px;
    ">COVID-19 Data Visualization</h2>

# Import libraries

In [ ]:
#!pip install plotly

In [ ]:
import numpy as np 
import pandas as pd 
import plotly as py
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Load data

Novel Corona Virus 2019 dataset ([Kaggle](https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset))

In [ ]:
# Read Data
df = pd.read_csv('../input/novel-corona-virus-2019-dataset/covid_19_data.csv')
df.head()

In [ ]:
# Rename columns
df = df.rename(columns={'Country/Region':'Country'})
df = df.rename(columns={'ObservationDate':'Date'})

# Rename 'Mainland China'
df['Country'] = df['Country'].replace('Mainland China', 'China')

# Create new column with active cases
df['Active'] = df['Confirmed'] - df['Deaths'] - df['Recovered']

df.head()

# COVID-19 in the world

In [ ]:
# Get last update
last_update = df[df['Date'] == max(df['Date'])].reset_index()
table_data = last_update.groupby(["Date"])[["Confirmed","Active","Recovered","Deaths"]].sum().reset_index()

# Create table
labels = ["Last Update","Confirmed","Active","Recovered","Deaths"]
fig = go.Figure(data=[go.Table(header=dict(values=labels),
                 cells=dict(values=table_data.loc[0,["Date","Confirmed","Active","Recovered","Deaths"]]))
                     ])

fig.update_layout(
    title_text='COVID-19 in the World: ',
    title_x = 0.5
)

fig.show()

# Proportion of Confirmed Cases by Country

This pie plot is also useful to visualize the proportion of active cases, recovered cases and deaths by country.

In [ ]:
# Create dataframe for pie plot
df_countries = df.groupby(['Country', 'Date']).sum().reset_index().sort_values('Date', ascending=False)
df_countries = df_countries.drop_duplicates(subset = ['Country'])
df_countries = df_countries[df_countries['Confirmed']>0]

# Create visualization
fig = px.pie(df_countries, values = 'Confirmed',names='Country', height=600)
fig.update_traces(textposition='inside', textinfo='percent+label')

fig.update_layout(
    title_text = 'Proportion of Confirmed Cases by Country',
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))

fig.show()

# Proportion of Recovered Cases by Country

The tree map is similar to a pie plot, it also helps visualize the proportion of confirmed, active, recovered cases and deaths.

In [ ]:
# Create visualization
fig = px.treemap(df_countries, path=['Country'], values='Recovered', height=600, width=1000)

fig.update_layout(
    title_text = 'Recovered Cases by Country',
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))

fig.show()

# World Distribution of the Confirmed Cases

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations = df_countries['Country'],
    locationmode = 'country names',
    z = df_countries['Confirmed'],
    colorscale = 'Reds',
    marker_line_color = 'black',
    marker_line_width = 0.5,
))

fig.update_layout(
    title_text = 'Confirmed Cases as of November 15, 2020',
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
        projection_type = 'equirectangular'
    )
)

# Global Spread of COVID-19

In [ ]:
# Create dataframe for animated cloropleth map
df_countrydate = df[df['Confirmed']>0]
df_countrydate = df_countrydate.groupby(['Date','Country']).sum().reset_index()

# Create visualization
fig = px.choropleth(df_countrydate, 
                    locations="Country", 
                    locationmode = "country names",
                    color="Confirmed", 
                    hover_name="Country", 
                    animation_frame="Date"
)

fig.update_layout(
    title_text = 'Global Spread of COVID-19',
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    )
)
    
fig.show()

# Most 10 Infected Countries

Same for confirmed, active and recovered cases.

In [ ]:
# Create datafram for bar charts
bar_data = df.groupby(['Country'])['Confirmed', 'Recovered', 'Active', 'Deaths'].sum().reset_index().sort_values('Confirmed', ascending=False).reset_index(drop=True)

# Create visualization
fig = px.bar(bar_data[0:10], 
             x="Country", 
             y="Confirmed",
             color='Country', 
             text = 'Confirmed', 
             orientation='v', height=600,
             title='Most 10 Infected Countries')
fig.show()

# Confirmed Cases, Recovered Cases and Death Over Time

In [ ]:
# Create dataframe for line graph
line_data = df.groupby('Date').sum().reset_index()

# Create visualization
line_data = line_data.melt(id_vars='Date', 
                           value_vars=['Confirmed', 'Recovered', 'Deaths'], 
                           var_name='Ratio', 
                           value_name='Value')

fig = px.line(line_data, 
              x="Date", y="Value", 
              color='Ratio', 
              title='Confirmed Cases, Recovered Cases, and Death Over Time')
fig.show()